In [ ]:

#max 27
#sonra devam et
#env değiştirildi. 
#best snakes so far tmp/Actor_v2.0-copy1-savepoint3 tmp/Critic_v2.0-copy1-savepoint3

In [1]:
import numpy as np
import import_ipynb
from envwithheart import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

env=environement()
info={}
env.env_init(info)

class a2c_v2:
    def __init__(self, info={}):
        self.n_actions = info.get("n_actions",4) 
        self.input_shape=info.get("input_shape",(12,12,2)) 
        self.EPISODES = info.get("episodes",100000) 
        self.lr = info.get("lr",0.001) 
        self.gamma=info.get("gamma",0.99)
        self.Actor, self.Critic = self.create_model(input_shape=self.input_shape, n_actions=self.n_actions, lr=self.lr)
        self.states, self.actions, self.rewards = [], [], []
        
    def create_model(self,input_shape,n_actions,lr):
        inputs= keras.Input(shape=input_shape)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(inputs)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(x)
        block_1_output = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(block_1_output)
        x = layers.Conv2D(64, 3, activation="relu",padding="same")(x)
        x = layers.add([x, block_1_output])
        x=layers.Flatten()(x)
        action = layers.Dense(n_actions, activation="softmax")(x)
        value = layers.Dense(1)(x)

        Actor = keras.Model(inputs = inputs, outputs = action)
        Actor.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=lr))

        Critic = keras.Model(inputs = inputs, outputs = value)
        Critic.compile(loss='mse', optimizer=keras.optimizers.RMSprop(lr=lr))

        return Actor, Critic
    def remember(self, state, action, reward):
        self.states.append(np.reshape(state,(1,12,12,2)))
        action_onehot = np.zeros([self.n_actions])
        action_onehot[action] = 1
        self.actions.append(action_onehot)
        self.rewards.append(reward)
    def act(self, state):
        prediction = self.Actor.predict(np.reshape(state,(1,12,12,2)))[0]
        #print(prediction)
        action = np.random.choice(self.n_actions, p=prediction)
        return action
    def discount_rewards(self, reward,states):
        gamma=0.01*env.length    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward,dtype=float)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add
        return discounted_r

        
    def replay(self):
        states = np.vstack(self.states)
        actions = np.vstack(self.actions)
        discounted_r = self.discount_rewards(self.rewards,states)
        values = self.Critic.predict(states)[:, 0]
        advantages = discounted_r - values
        # training Actor and Critic networks
        self.Actor.fit(states, actions, sample_weight=advantages, epochs=1, verbose=0)
        self.Critic.fit(states, discounted_r, epochs=1, verbose=0)
        self.states, self.actions, self.rewards = [], [], []
    def step(self, action):
        reward, next_state, done,score = env.step(action)
        #print(next_state[:,:,0])
        return next_state, reward, done,score
    def run(self):
        for e in range(self.EPISODES):
            state = env.start()
            done = False
            while not done:
                action = self.act(state)
                next_state, reward, done,score= self.step(action)
                self.remember(state, action, reward)
                state = next_state
                if done:
                    print("Episode :",e," Score :",score)
                    if(e%1000==0):
                        self.save()
                    self.replay()
    def save(self):
        self.Actor.save('tmp/Actorstardeneme')
        self.Critic.save('tmp/Criticstardeneme')
    def load(self):
        self.Actor = tf.keras.models.load_model('tmp/Actorstardeneme')
        self.Critic = tf.keras.models.load_model('tmp/Criticstardeneme')


importing Jupyter notebook from envwithheart.ipynb
-0.1
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [2]:
a=a2c_v2()
a.Critic.summary()
a.load()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 12, 2)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 12, 12, 64)   1216        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 64)   36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 6, 6, 64)     0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [ ]:
a.run()

Episode : 0  Score : 16
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tmp/Actorstardeneme\assets
INFO:tensorflow:Assets written to: tmp/Criticstardeneme\assets
Episode : 1  Score : 12
Episode : 2  Score : 0
Episode : 3  Score : 0
Episode : 4  Score : 0
Episode : 5  Score : 0
Episode : 6  Score : 0
Episode : 7  Score : 0
Episode : 8  Score : 0
Episode : 9  Score : 0
Episode : 10  Score : 0
Episode : 11  Score : 0
Episode : 12  Score : 0
Episode : 13  Score : 0
Episode : 14  Score : 0
Episode : 15  Score : 0
Episode : 16  Score : 0
Episode : 17  Score : 0
Episode : 18  Score : 0
Episode : 19  Score : 0
Episode : 20  Score : 0
Episode : 21  Score : 0
Episode : 22  Score : 0
Episode : 23  Score : 0
Episode : 24  Score : 0
Episode : 25  Score : 0
Episode : 26  Score : 0
E

In [ ]:
a.save()